# Analysis of Hook Errors in a Distance 3 Surface Code

This notebook analyzes the effect of CNOT gate ordering on the propagation of hook errors in a d=3 rotated surface code. It uses the `zx_error_prop` library to:

1. Generate surface code circuits with different CNOT orderings.
2. Inject a representative set of hook errors into each circuit.
3. Propagate the errors using PyZX's `gflow` functionality.
4. Count the number of resulting logical X and Z errors for each CNOT permutation.

The goal is to identify CNOT orderings that are more robust against this type of error.

In [ ]:
# === 1. Setup and Imports ===
import itertools
import pprint

import pyzx as zx
import matplotlib.pyplot as plt

from zx_error_prop.surface_code import generate_surface_code_qasm, plot_rotated_code
from zx_error_prop.propagation import propagate_pauli_error, get_output_errors, count_logical_errors

In [ ]:
# === 2. Define Experiment Parameters ===

DISTANCE = 3

# Define the logical operators by the qubit indices they cross at the output.
# These are for the d=3 code.
Z_LOGICALS = [[73, 74, 75], [76, 77, 78], [79, 80, 81]]
X_LOGICALS = [[73, 76, 79], [74, 77, 80], [75, 78, 81]]

# Define the set of hook errors to test.
# Each error is defined by a tuple: ((source_vertex, target_vertex), 'ErrorType')
HOOK_ERRORS = [
    ((27, 25), 'Z'), 
    ((37, 35), 'X'), 
    ((55, 53), 'X'), 
    ((63, 61), 'Z')
]

# Plot the code layout for visualization
plot_rotated_code(DISTANCE)
plt.show()

In [ ]:
# === 3. Run the Analysis ===

print("Starting analysis... This may take a few minutes.")

# Generate all 4! = 24 permutations of CNOT orderings
permutations = list(itertools.permutations([0, 1, 2, 3]))
analysis_results = {}

for i, perm in enumerate(permutations):
    print(f"Processing permutation {i+1}/{len(permutations)}: {perm}")
    perm_list = list(perm)
    
    # Define the custom CNOT ordering for the four central, weight-4 stabilizers.\n    # To test different orderings for different plaquettes, you can modify this\n    # dictionary. For example:\n    # custom_orders = {\n    #     'Z0': [0, 1, 2, 3],\n    #     'X1': [3, 2, 1, 0],\n    #     'X2': [1, 2, 3, 0],\n    #     'Z3': perm_list\n    # }\n    custom_orders = {
        'Z0': perm_list,
        'X1': perm_list,
        'X2': perm_list,
        'Z3': perm_list
    }
    
    # Generate the QASM string and convert to a ZX-graph
    qasm_string = generate_surface_code_qasm(DISTANCE, custom_cnot_orderings=custom_orders)
    graph = zx.sqasm(qasm_string, simplify=False)
    
    total_z_errors = 0
    total_x_errors = 0
    
    # For each permutation, test every hook error
    for hook in HOOK_ERRORS:
        # Propagate the error and get the final errors on the output qubits
        final_web, _ = propagate_pauli_error(graph, [hook])
        output_errors = get_output_errors(graph, final_web)
        
        # Count the number of logical errors that occurred
        z_errs, x_errs = count_logical_errors(output_errors, Z_LOGICALS, X_LOGICALS)
        
        total_z_errors += z_errs
        total_x_errors += x_errs
        
    analysis_results[perm] = (total_z_errors, total_x_errors)

print("\nAnalysis complete.")

In [ ]:
# === 4. Display Results ===

print("--- CNOT Permutation Analysis Results ---")
print("Format: (Permutation): (Total Z Errors, Total X Errors)\n")

# Sort results by the total number of errors (Z+X), ascending
sorted_results = sorted(analysis_results.items(), key=lambda item: sum(item[1]))

for perm, errors in sorted_results:
    print(f"{str(perm):<15}: {errors}")

print("\n--- Summary ---")
good_permutations = [p for p, e in sorted_results if sum(e) == 0]
print(f"Found {len(good_permutations)} permutations that resulted in 0 logical errors:")
for p in good_permutations:
    print(p)